In [30]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from IPython.display import display
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

In [31]:
spark = SparkSession.builder\
                    .appName('Titanic')\
                    .getOrCreate()

In [32]:
titanic_data = spark.read.csv('Titanic_data.csv', header=True, inferSchema=True)

In [33]:
titanic_data.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [34]:
titanic_data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [35]:
# Finding maximum fare - another way
titanic_data.agg(F.max(titanic_data.Fare)).collect()

[Row(max(Fare)=512.3292)]

In [36]:
titanic_data.groupBy('Pclass').count()

DataFrame[Pclass: int, count: bigint]

In [37]:
titanic_data.groupBy('Pclass').avg('Age').collect()

[Row(Pclass=1, avg(Age)=38.233440860215055),
 Row(Pclass=3, avg(Age)=25.14061971830986),
 Row(Pclass=2, avg(Age)=29.87763005780347)]

In [38]:
#Descending order of age
titanic_data.orderBy(titanic_data.Age.desc())

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

In [39]:
titanic_data.select("Survived","Pclass","Embarked").show(10)

+--------+------+--------+
|Survived|Pclass|Embarked|
+--------+------+--------+
|       0|     3|       S|
|       1|     1|       C|
|       1|     3|       S|
|       1|     1|       S|
|       0|     3|       S|
|       0|     3|       Q|
|       0|     1|       S|
|       0|     3|       S|
|       1|     3|       S|
|       1|     2|       C|
+--------+------+--------+
only showing top 10 rows



In [40]:
titanic_data.groupBy("Survived").count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



#### check Null Values

In [41]:
def null_value_count(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
        temp = k,nullRows
        null_columns_counts.append(temp)
    return(null_columns_counts)

In [42]:
null_columns_count_list = null_value_count(titanic_data)

In [43]:
spark.createDataFrame(null_columns_count_list, ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|              Embarked|                2|
+----------------------+-----------------+

